In [14]:
import sys
import pandas as pd

#Importing Drive (Only for colab)
runningOnColab = 'google.colab' in sys.modules
if runningOnColab:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd '/content/drive/Shareddrives/Capstone/quality-controlled-paraphrase-generation'



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/Capstone/quality-controlled-paraphrase-generation


In [16]:
import pandas as pd
import os
cwd = os.getcwd()
os.environ['PYTHONPATH'] = cwd

# New Section

In [11]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
!pip install clearml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Prepare Data

In [13]:
!python QCPG/evaluate.py \
--train_file data/mscoco/train.csv.gz \
--dataset_split train \
--predictions_column source \
--references_column target \
--metric metrics/para_metric \
--output_path new_data/mscoco/train.csv.gz

Using custom data configuration default-e0c679be8369f11f
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-e0c679be8369f11f/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)
2022-11-10 02:04:49.013545: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!
Computing metric...
bleu: 100% 10/10 [00:00<00:00, 310.85it/s]
syntdiv:calc_dist: 100% 10/10 [00:00<00:00, 39.68it/s]
phonemetric:find_sim: 100% 10/10 [00:02<00:00,  4.08it/s]
morphmetric:find_sim: 100% 10/10 [00:00<00:00, 61.10it/s]


In [5]:
!python QCPG/evaluate.py \
--train_file data/mscoco/validation.csv.gz \
--dataset_split train \
--predictions_column source \
--references_column target \
--metric metrics/para_metric \
--output_path new_data/mscoco/validation.csv.gz

Using custom data configuration default-9af5fcb93faffdeb
0 tables [00:00, ? tables/s]/usr/local/lib/python3.7/dist-packages/tqdm/std.py:1133: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  for obj in iterable:
/usr/local/lib/python3.7/dist-packages/tqdm/std.py:1133: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  for obj in iterable:
Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9af5fcb93faffdeb/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.
2022-11-10 01:48:50.867162: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!
Traceback (most recent call last):
  File "Q

In [6]:
!python QCPG/evaluate.py \
--train_file data/mscoco/test.csv.gz \
--dataset_split train \
--predictions_column source \
--references_column target \
--metric metrics/para_metric \
--output_path new_data/mscoco/test.csv.gz

Using custom data configuration default-bb2bde7c7bb15efd
0 tables [00:00, ? tables/s]/usr/local/lib/python3.7/dist-packages/tqdm/std.py:1133: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  for obj in iterable:
/usr/local/lib/python3.7/dist-packages/tqdm/std.py:1133: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  for obj in iterable:
Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-bb2bde7c7bb15efd/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.
2022-11-10 01:49:10.959191: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!
Traceback (most recent call last):
  File "Q

In [7]:
!python QCPG/train.py --model_name_or_path t5-base \
--do_train --do_eval --source_column reference \
--target_column prediction --per_device_eval_batch_size 16 \
--per_device_train_batch_size 16 --predict_with_generate \
--evaluation_strategy epoch --num_train_epochs 6 \
--lr_scheduler_type constant --save_total_limit 1 \
--dataset_generate_mode force_redownload --dataset_keep_in_memory \
--conditions_columns '["semantic_sim", "lexical_div", "syntactic_div", "phonological_div", "morphological_div"]' \
--overwrite_output_dir \
--dataset_map 'semantic_sim = 5 * round(bleurt_score * 100 / 5); lexical_div = 5 * round(set_diversity * 100 / 5); syntactic_div = 5 * round(syn_diversity * 100 / 5); phonological_div = 5 * round(phon_diversity * 100 / 5); morphological_div = 5 * round(morph_diversity * 100 / 5);' \
--train_file new_data/mscoco/train.csv.gz \
--validation_file new_data/mscoco/validation.csv.gz \
--learning_rate 1e-3 \
--output_dir new_data/t5-base-cond-mscoco-bleurt-lr1e-3-v1 \
--dataset_generate_mode force_redownload


INFO:__main__:Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='new_data/t5-base-cond-mscoco-bleurt-lr1e-3-v1', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.EPOCH: 'epoch'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=0.001, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=6.0, max_steps=-1, lr_scheduler_type=<SchedulerType.CONSTANT: 'constant'>, warmup_ratio=0.0, warmup_steps=0, logging_dir='runs/Nov10_01-49-27_a8ba82c87e0e', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=500, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=500, save_total_limit=1, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', fp1

KeyboardInterrupt: ignored

In [ ]:
!python QCPG/predict.py \
--per_device_eval_batch_size 256 --per_device_train_batch_size 256 \
--source_column reference --target_column prediction \
--conditions_columns '["semantic_sim", "lexical_div", "syntactic_div", "phonological_div", "morphological_div"]' \
--dataset_map 'semantic_sim = 5 * round(bleurt_score * 100 / 5); lexical_div = 5 * round(set_diversity * 100 / 5); syntactic_div = 5 * round(syn_diversity * 100 / 5); phonological_div = 5 * round(phon_diversity * 100 / 5); morphological_div = 5 * round(morph_diversity * 100 / 5);' \
--train_file new_data/mscoco/validation.csv.gz \
--dataset_split train \
--model_name_or_path new_data/t5-base-cond-mscoco-bleurt-lr1e-3-v1 \
--output_dir new_data/validation/t5-base-cond-mscoco-bleurt-lr1e-3-v1